In [1]:
from supabase import create_client
from dotenv import load_dotenv
import os, time
import matplotlib as plt
from ib_insync import *
import financedatabase as fd

import pandas as pd
import datetime as dt
from shared_resources import add_log, start_event, connect_to_IB
from strategies.strategy1 import Strategy

load_dotenv()

# Initialize Supabase client
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)



PARAMS = {
    1:{'name':'Monthly Trendfilter','value': 10,
       'description':"The 10M SMA Trendfilter is used as a sell signal if the price drops below."},
    2:{'name':"Structural Trendfilter",'value':50,
       'description':"""This filter is used to re-enter the market if the price is below the monthly trendfilter and was below the structural trend, but price just crossed this structural trendline from below."""},
    3:{'name':'Equity Weight','value':30,'description':'Weight for equity allocation'},
    4:{'name':'Fixed Income Weight','value':90,'description':'Weight for FI allocation'},
}

2023-11-27 06:11:49,490:INFO - NumExpr defaulting to 8 threads.


In [2]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=2)
#ib = connect_to_IB()

2023-11-27 06:13:00,676:INFO - Connecting to 127.0.0.1:7497 with clientId 2...
2023-11-27 06:13:00,678:INFO - Connected
2023-11-27 06:13:00,773:INFO - Logged on to server version 176
2023-11-27 06:13:00,781:INFO - API connection ready
2023-11-27 06:13:00,790:INFO - Warning 2104, reqId -1: Verbindung zum Marktdatenzentrum ist OK:usfarm.nj
2023-11-27 06:13:00,791:INFO - Warning 2104, reqId -1: Verbindung zum Marktdatenzentrum ist OK:eufarm
2023-11-27 06:13:00,792:INFO - position: Position(account='DU3180649', contract=Stock(conId=239363241, symbol='IUSQ', exchange='IBIS2', currency='EUR', localSymbol='IUSQ', tradingClass='ETF'), position=1.0, avgCost=67.6)
2023-11-27 06:13:00,792:INFO - Warning 2104, reqId -1: Verbindung zum Marktdatenzentrum ist OK:cashfarm
2023-11-27 06:13:00,793:INFO - Warning 2104, reqId -1: Verbindung zum Marktdatenzentrum ist OK:usfarm
2023-11-27 06:13:00,793:INFO - Warning 2104, reqId -1: Verbindung zum Marktdatenzentrum ist OK:eufarmnj
2023-11-27 06:13:00,794:INF

<IB connected to 127.0.0.1:7497 clientId=2>

In [3]:
# Initialize the Equities database
equities = fd.Equities()
# Find all US stocks
us_stocks = equities.select(country="United States", exclude_exchanges=False)
# Filter to domestic stocks & US Exchange Listings only with active ISINs
us_stocks = us_stocks[(us_stocks['currency']=='USD') & (~us_stocks.index.str.contains('\\.'))].dropna(subset=['isin']).dropna(subset=['cusip'])
us_stocks.tail()

,name,summary,currency,sector,industry_group,industry,exchange,market,country,state,city,zipcode,website,market_cap,isin,cusip,figi,composite_figi,shareclass_figi
symbol,,,,,,,,,,,,,,,,,,,
ZNOG,"Zion Oil & Gas, Inc.","Zion Oil & Gas, Inc., together with its subsid...",USD,Energy,Energy,"Oil, Gas & Consumable Fuels",PNK,OTC Bulletin Board,United States,TX,Dallas,75243,http://www.zionoil.com,Micro Cap,US9896961094,989696109,NaN,NaN,NaN
ZOM,Zomedica Corp.,"Zomedica Corp., a development stage veterinary...",USD,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,ASE,NYSE MKT,United States,MI,Ann Arbor,48108,http://zomedica.com,Micro Cap,US98980F1049,98980F104,NaN,NaN,NaN
ZS,"Zscaler, Inc.","Zscaler, Inc. operates as a cloud security com...",USD,Information Technology,Software & Services,Software,NMS,NASDAQ Global Select,United States,CA,San Jose,95134,http://www.zscaler.com,Large Cap,US98980G1022,98980G102,BBG00K4MD5L3,BBG003338H34,BBG003338H61
ZUMZ,Zumiez Inc.,"Zumiez Inc., together with its subsidiaries, o...",USD,Consumer Discretionary,Consumer Durables & Apparel,"Textiles, Apparel & Luxury Goods",NMS,NASDAQ Global Select,United States,WA,Lynnwood,98036,http://www.zumiez.com,Small Cap,US9898171015,989817101,BBG000PYXPR6,BBG000PYX812,BBG001SGPKJ9
ZYNE,"Zynerba Pharmaceuticals, Inc.","Zynerba Pharmaceuticals, Inc. operates as a cl...",USD,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,NGM,Nordic Growth Market,United States,PA,Devon,19333,http://zynerba.com,Micro Cap,US98986X1090,98986X109,NaN,NaN,NaN


In [4]:
symbols = us_stocks.index.unique()
contracts = [Stock(con,'SMART','USD') for con in symbols]
contracts = ib.qualifyContracts(*contracts)



2023-11-27 06:13:21,496:ERROR - Error 200, reqId 12: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='ABC', exchange='SMART', currency='USD')
2023-11-27 06:13:21,915:ERROR - Error 200, reqId 35: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='ADMP', exchange='SMART', currency='USD')
2023-11-27 06:13:22,015:ERROR - Error 200, reqId 53: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='AGFS', exchange='SMART', currency='USD')
2023-11-27 06:13:22,084:ERROR - Error 200, reqId 57: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='AHPI', exchange='SMART', currency='USD')
2023-11-27 06:13:22,089:ERROR - Error 200, reqId 58: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='AIABF', exchange='SMART', currency='USD')
2023-11-27 06:13:22,091:ERROR - Error 200, reqId 56: Es wurde keine Wertpapierdefinition zu der Anfrage g

# Query Database and calculate some moving averages & apply filters


In [131]:
def fetch_data(table, offset=0, limit=1000):
    fetched_data = supabase.table(table).select("*").range(offset, offset + limit - 1).execute()
    return fetched_data.data

all_data = []
offset = 0
limit = 1000  # Adjust the limit based on your needs

while True:
    data = fetch_data("prices", offset, limit)
    if not data:
        break
    all_data.extend(data)
    offset += limit

df = pd.DataFrame(all_data)



2023-11-27 15:45:57,920:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,012:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,101:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,191:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,284:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,366:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,447:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices?select=%2A "HTTP/1.1 200 OK"
2023-11-27 15:45:58,527:INFO - HTTP Request: GET https://mjuqc

In [133]:
supabase_data = df

In [144]:
df = supabase_data

In [146]:
df

,id,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
date,,,,,,,,,,,,
2022-11-25,16,154.00,155.86,153.62,155.86,3312.52,A,NaN,NaN,None,NaN,155.86
2022-11-28,17,154.78,155.75,150.67,151.23,7205.51,A,NaN,NaN,None,NaN,155.86
2022-11-29,18,150.81,151.83,149.43,149.88,4845.70,A,NaN,NaN,None,NaN,155.86
2022-11-30,19,150.32,153.89,148.05,153.89,9364.18,A,NaN,NaN,None,NaN,155.86
2022-12-01,20,155.63,155.89,153.34,155.02,7911.36,A,NaN,NaN,None,NaN,155.86
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-17,364047,18.11,18.49,17.88,18.05,1128.04,ZUMZ,16.9164,18.23275,None,0.841428,28.20
2023-11-20,364048,17.98,18.52,17.83,18.42,1003.23,ZUMZ,16.9358,18.18445,None,0.830612,28.20
2023-11-21,364049,18.19,18.46,17.94,18.24,931.75,ZUMZ,16.9710,18.14270,None,0.808425,28.20


In [147]:
#df.set_index('date',drop=True,inplace=True)
df = df.sort_values(by='symbol',axis='index',kind='stable')
df = df.groupby('symbol').apply(lambda group: group.iloc[-1::])
df = df[df.volume >= 10e2]
df


,,id,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
symbol,date,,,,,,,,,,,,
A,2023-11-24,266,123.9200,126.7700,123.4800,126.6200,7128.42,A,110.600800,123.662650,None,3.371811,158.1800
AA,2023-11-24,517,26.1600,26.5600,26.1600,26.2400,9693.48,AA,26.313200,34.217800,None,0.998431,54.8700
AAL,2023-11-24,768,12.3900,12.4800,12.2500,12.3100,131995.07,AAL,12.120000,14.509300,None,0.394139,18.8000
AAOI,2023-11-24,1270,13.2500,13.6299,12.9200,13.0400,7672.52,AAOI,9.475200,6.435425,None,0.960665,15.4800
AAP,2023-11-24,1772,52.3300,53.3400,52.1200,53.0800,5754.82,AAP,53.822800,86.182300,None,2.737440,153.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIP,2023-11-24,362795,12.8400,12.9700,12.7700,12.8000,1263.32,ZIP,11.843600,15.636900,None,0.423586,23.8100
ZM,2023-11-24,363047,64.0300,64.9800,63.8200,64.5300,11180.74,ZM,64.477600,67.857450,None,1.920055,84.6600
ZNOG,2023-11-24,363298,0.0570,0.0595,0.0538,0.0574,5782.48,ZNOG,0.068156,0.076678,None,0.007824,0.1220


In [142]:
df

,,id,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
symbol,date,,,,,,,,,,,,
AAL,2023-11-24,768,12.3900,12.480,12.2500,12.3100,131995.07,AAL,12.120000,14.509300,None,0.394139,18.8000
AAPL,2023-11-24,2023,190.8700,190.900,189.2500,189.9700,168695.54,AAPL,177.881200,173.900300,None,2.783318,195.9300
ABNB,2023-11-24,3278,129.0500,129.590,127.7800,128.3700,10851.36,ABNB,126.233200,125.736450,None,4.353070,153.3300
ABT,2023-11-24,3529,102.6800,103.180,102.1900,102.8700,13353.17,ABT,96.238200,102.165300,None,1.864296,113.9900
AEO,2023-11-24,10552,16.8500,16.970,16.3400,16.6500,28217.39,AEO,17.130200,14.355700,None,0.857929,19.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XERS,2023-11-24,358779,1.7300,1.950,1.7300,1.9300,10554.07,XERS,1.770900,2.085200,None,0.176402,2.9800
YUMC,2023-11-24,361791,44.9300,45.250,44.6500,44.6700,18357.28,YUMC,51.149400,56.801850,None,1.377786,64.0600
ZI,2023-11-24,362544,14.1700,14.280,14.0100,14.1700,21683.38,ZI,15.325400,21.133000,None,0.561408,31.3600


In [148]:
# Initialize a column for tracking whether a stock dipped below its 50D_MA
df['dipped_below_50D_MA'] = False


# Loop through the DataFrame and check each row
for index, row in df.iterrows():
    # Get the slice of the last 20 days up to the current row
    window = df.loc[:index].tail(20)

    # Check if the stock dipped below its 50D_MA in this window
    if (window['close'] < window['50D_MA']).any():
        df.at[index, 'dipped_below_50D_MA'] = True

# Apply your initial filters
filtered_df = df[
    (df['close'] > df['200D_MA']) &
    (df['volume'] >= 1e5) &
    (df['close'] > df['50D_MA']) &
    (df['close'] >= 0.96 * df['52W_High']) & 
    (df['close'] <= 1.04 * df['52W_High']) &
    (df['dipped_below_50D_MA'] == True)
]

filtered_df

,,id,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High,dipped_below_50D_MA
symbol,date,,,,,,,,,,,,,
AAPL,2023-11-24,2023,190.87,190.9,189.25,189.97,168695.54,AAPL,177.8812,173.90030,None,2.783318,195.93,True
AMZN,2023-11-24,24606,146.67,147.2,145.32,146.74,162271.36,AMZN,133.9128,121.24645,None,3.502042,146.74,True


In [139]:
# Filter for symbols above their 200-day MA
filtered_df = df[(df['close'] > df['200D_MA']) & (df['close'] > df['50D_MA'])]
# Filter for symbols that are 1-4% away from their 52-week high
filtered_df = filtered_df[(filtered_df['close'] >= 0.96 * filtered_df['52W_High']) & (filtered_df['close'] <= 1.04 * filtered_df['52W_High'])]
filtered_df

,,id,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
symbol,date,,,,,,,,,,,,
AAPL,2023-11-24,2023,190.87,190.90,189.25,189.97,168695.54,AAPL,177.8812,173.90030,None,2.783318,195.93
AMZN,2023-11-24,24606,146.67,147.20,145.32,146.74,162271.36,AMZN,133.9128,121.24645,None,3.502042,146.74
ARCC,2023-11-24,31633,19.88,19.92,19.78,19.89,10736.23,ARCC,19.2992,18.36075,None,0.205385,19.89
COIN,2023-11-24,84157,108.46,116.94,107.98,115.54,85502.08,COIN,82.1504,73.00980,None,5.833905,115.54
DASH,2023-11-24,100263,93.84,94.88,93.07,93.81,16517.86,DASH,81.2478,72.81080,None,3.145859,95.75
DELL,2023-11-24,103595,74.63,75.05,74.17,74.41,11505.16,DELL,69.1406,53.21485,None,1.510625,74.68
DKNG,2023-11-24,106607,38.60,39.13,38.46,38.70,32819.37,DKNG,31.1068,25.91115,None,1.423939,38.70
IBM,2023-11-24,164809,155.13,155.40,153.92,155.18,10358.01,IBM,143.6776,133.47925,None,1.951933,155.18
INTC,2023-11-24,170832,43.68,44.00,43.61,43.96,92550.52,INTC,37.0508,32.84690,None,1.181126,44.74


In [130]:

df

,,id,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
symbol,date,,,,,,,,,,,,
AAL,2023-11-24,768,12.3900,12.4800,12.2500,12.3100,131995.07,AAL,12.120000,14.509300,None,0.394139,18.80
AAPL,2023-11-24,2023,190.8700,190.9000,189.2500,189.9700,168695.54,AAPL,177.881200,173.900300,None,2.783318,195.93
AMD,2023-11-24,19587,122.0300,123.2500,121.5400,122.3100,154773.36,AMD,106.619400,103.247700,None,3.763794,129.19
AMZN,2023-11-24,24606,146.6700,147.2000,145.3200,146.7400,162271.36,AMZN,133.912800,121.246450,None,3.502042,146.74
MARA,2023-11-24,203708,11.0400,11.6000,10.8500,11.4100,386513.57,MARA,8.922400,10.361200,None,0.875330,19.30
PLTR,2023-11-24,260167,19.3800,19.6000,18.9300,19.2000,351650.39,PLTR,17.009200,13.452700,None,1.008945,21.34
PLUG,2023-11-24,260418,3.4300,3.4900,3.3600,3.4700,150232.62,PLUG,6.259400,9.413550,None,0.494863,17.89
RIOT,2023-11-24,277476,11.6500,12.2600,11.4177,12.0200,163828.59,RIOT,9.996800,11.026275,None,0.912117,20.29
SEEL,2023-11-24,292027,0.1615,0.1615,0.1200,0.1309,101934.55,SEEL,0.221900,0.790542,None,0.026771,1.64


In [125]:
# Group by 'symbol', filter where 'close' > '50D_MA', and then rank by 'ATR'
filtered_df = df.groupby('symbol').filter(lambda x: (x['close'] > x['50D_MA']).any())

# Get the latest entry for each symbol
latest_entry_per_symbol = filtered_df.groupby('symbol').last()

# Sort the DataFrame by 'ATR' in ascending order
sorted_latest_entries = latest_entry_per_symbol.sort_values(by='ATR', ascending=True)
sorted_latest_entries

ValueError: 'symbol' is both an index level and a column label, which is ambiguous.

In [123]:
today = dt.datetime.today().isoformat()[:10]
today




def get_last_trading_day():
    today = dt.datetime.today()
    # If it's Sunday (6) or Monday (0), return last Friday's date
    if today.weekday() == 0:
        trading_day = today  - dt.timedelta(days=3)
    elif today.weekday() == 6:  # Sunday
        trading_day = today - dt.timedelta(days=2)
    else:  # Weekday
        trading_day = today  - dt.timedelta(days=1)

    return trading_day

day = get_last_trading_day()
lbd = day.date().isoformat()
# filtered_df = df.groupby('symbol').filter(lambda x: (x['close'] > x['200D_MA']).any())

# filtered_df = filtered_df[filtered_df["volume"]>10e4]
# filtered_df = filtered_df.groupby('symbol').apply(lambda group: group.iloc[-1::]).sort_values('ATR',ascending=False)

# filtered_df


In [95]:
import yfinance as yf
data = yf.download(['AAPL', 'TSLA'], group_by="Ticker", start="2022-12-31")

[*********************100%%**********************]  2 of 2 completed


In [96]:
ydf = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
ydf

,Ticker,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2023-01-03,AAPL,130.279999,130.899994,124.169998,125.070000,124.374802,112117500
2023-01-03,TSLA,118.470001,118.800003,104.639999,108.099998,108.099998,231402800
2023-01-04,AAPL,126.889999,128.660004,125.080002,126.360001,125.657639,89113600
2023-01-04,TSLA,109.110001,114.589996,107.519997,113.639999,113.639999,180389000
2023-01-05,AAPL,127.129997,127.769997,124.760002,125.019997,124.325081,80962700
...,...,...,...,...,...,...,...
2023-11-21,TSLA,235.039993,243.619995,233.339996,241.199997,241.199997,122288000
2023-11-22,AAPL,191.490005,192.929993,190.830002,191.309998,191.309998,39617700
2023-11-22,TSLA,242.039993,244.009995,231.399994,234.210007,234.210007,117950600


In [ ]:
d

# HDF5 Tests - will likely move to storing financial data locally

In [64]:
# Save the DataFrame to an HDF5 file
df.to_hdf('financial_data.h5', key='prices', mode='w')

C:\Users\local_Johannes.Lange1\Temp\ipykernel_34228\3596988273.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['symbol', '10M_MA'], dtype='object')]

  df.to_hdf('financial_data.h5', key='prices', mode='w')


In [67]:
retrieved_df = pd.read_hdf('financial_data.h5')
retrieved_df

,id,date,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
0,16,2022-11-25,154.00,155.86,153.62,155.86,3312.52,A,NaN,NaN,None,NaN,155.86
1,17,2022-11-28,154.78,155.75,150.67,151.23,7205.51,A,NaN,NaN,None,NaN,155.86
2,18,2022-11-29,150.81,151.83,149.43,149.88,4845.70,A,NaN,NaN,None,NaN,155.86
3,19,2022-11-30,150.32,153.89,148.05,153.89,9364.18,A,NaN,NaN,None,NaN,155.86
4,20,2022-12-01,155.63,155.89,153.34,155.02,7911.36,A,NaN,NaN,None,NaN,155.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363166,364047,2023-11-17,18.11,18.49,17.88,18.05,1128.04,ZUMZ,16.9164,18.23275,None,0.841428,28.20
363167,364048,2023-11-20,17.98,18.52,17.83,18.42,1003.23,ZUMZ,16.9358,18.18445,None,0.830612,28.20
363168,364049,2023-11-21,18.19,18.46,17.94,18.24,931.75,ZUMZ,16.9710,18.14270,None,0.808425,28.20
363169,364050,2023-11-22,18.29,18.84,18.26,18.43,829.71,ZUMZ,17.0108,18.10150,None,0.793538,28.20


In [34]:
# Calculate the date 20 days ago
twenty_days_ago = (dt.datetime.now() - dt.timedelta(days=20)).strftime('%Y-%m-%d')

# Ensure that 'date' and '50D_MA' are in the correct format
df['date'] = pd.to_datetime(df['date'])

# The '50D_MA' should be a numeric type (float). Convert it if it's not.
df['50D_MA'] = pd.to_numeric(df['50D_MA'], errors='coerce')

# Now perform the filtering
filtered_df = df[(df['close'] > df['50D_MA']) & (df['date'] >= pd.to_datetime(twenty_days_ago))]

# Sort the results
filtered_df = filtered_df.sort_values(by='date', ascending=False)

filtered_df

,id,date,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
363170,364051,2023-11-24,18.35,18.470,17.6900,18.30,842.41,ZUMZ,17.034000,18.063750,None,0.792571,28.20
166134,166566,2023-11-24,28.74,28.890,28.5000,28.61,240.80,IDT,26.784600,28.240250,None,0.636203,34.77
55992,56309,2023-11-24,3.75,4.130,3.7500,3.89,26741.60,BLUE,3.098200,3.720850,None,0.294349,8.49
166384,166817,2023-11-24,1.06,1.280,1.0600,1.26,50.70,IDXG,1.008782,1.323903,None,0.099840,2.91
338079,338935,2023-11-24,55.20,55.270,54.1200,54.97,183.38,UVV,48.194000,49.055100,None,1.038986,55.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76529,76870,2023-11-07,307.71,307.800,293.3000,295.03,21804.91,CI,293.383800,278.186100,None,7.243972,334.78
281684,282480,2023-11-07,96.13,96.370,95.4400,96.04,2144.75,RPM,95.346800,89.229700,None,1.876626,105.20
42853,43161,2023-11-07,216.92,219.020,214.7000,216.37,4783.21,AXON,207.675800,203.510500,None,5.675011,227.53
77528,77870,2023-11-07,1.99,2.085,1.9573,1.99,4196.48,CKPT,1.872000,2.925300,None,0.152519,7.73


In [43]:

data, count = supabase.table('prices, prices!inner (50D_MA)').select('symbol').filter('close', 'gt', 'prices.50D_MA').execute()


2023-11-27 07:46:32,138:INFO - HTTP Request: GET https://mjuqcwokjerpvhnmteco.supabase.co/rest/v1/prices,%20prices!inner%20(50D_MA)?select=symbol&close=gt.prices.50D_MA "HTTP/1.1 404 Not Found"


APIError: {'code': '42P01', 'details': None, 'hint': None, 'message': 'relation "public.prices, prices!inner (50D_MA)" does not exist'}

,id,date,open,high,low,close,volume,symbol,50D_MA,200D_MA,10M_MA,ATR,52W_High
0,16,2022-11-25,154.00,155.86,153.62,155.86,3312.52,A,NaN,NaN,None,NaN,155.86
1,17,2022-11-28,154.78,155.75,150.67,151.23,7205.51,A,NaN,NaN,None,NaN,155.86
2,18,2022-11-29,150.81,151.83,149.43,149.88,4845.70,A,NaN,NaN,None,NaN,155.86
3,19,2022-11-30,150.32,153.89,148.05,153.89,9364.18,A,NaN,NaN,None,NaN,155.86
4,20,2022-12-01,155.63,155.89,153.34,155.02,7911.36,A,NaN,NaN,None,NaN,155.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1011,2023-11-13,7.03,7.30,7.00,7.26,2357.84,AAN,9.1612,11.72015,None,0.378635,15.84
996,1012,2023-11-14,7.78,8.50,7.78,8.45,5406.98,AAN,9.1194,11.69360,None,0.440161,15.84
997,1013,2023-11-15,8.40,8.83,8.40,8.80,2326.70,AAN,9.0870,11.66675,None,0.439436,15.84
998,1014,2023-11-16,8.76,8.87,8.38,8.58,3347.55,AAN,9.0582,11.63765,None,0.443047,15.84


In [ ]:
symbols = us_stocks.index.unique()
contracts = [Stock(con,'SMART','USD') for con in symbols]
contracts = ib.qualifyContracts(*contracts)



2023-11-24 22:06:22,754:ERROR - Error 200, reqId 12: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='ABC', exchange='SMART', currency='USD')
2023-11-24 22:06:23,270:ERROR - Error 200, reqId 35: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='ADMP', exchange='SMART', currency='USD')
2023-11-24 22:06:24,171:ERROR - Error 200, reqId 57: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='AHPI', exchange='SMART', currency='USD')
2023-11-24 22:06:24,172:ERROR - Error 200, reqId 58: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='AIABF', exchange='SMART', currency='USD')
2023-11-24 22:06:24,173:ERROR - Error 200, reqId 53: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Stock(symbol='AGFS', exchange='SMART', currency='USD')
2023-11-24 22:06:24,177:ERROR - Error 200, reqId 56: Es wurde keine Wertpapierdefinition zu der Anfrage g

In [ ]:
df = util.df(data)

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head()

In [ ]:
for bar in df[-5:].iterrows():
    print(bar[1]['close'])

In [60]:
bars= util.tree(ib.reqHistoricalData(contract=con,endDateTime='',durationStr='1 Y',barSizeSetting='1 day',whatToShow='ADJUSTED_LAST',useRTH=True))

In [61]:
bars[-1]

{'BarData': {'date': '2023-11-24',
  'open': 123.92,
  'high': 126.77,
  'low': 123.48,
  'close': 126.62,
  'volume': 7128.42,
  'average': 125.665,
  'barCount': 5677}}

In [ ]:
for contract in contracts:
    try:
        bars = ib. reqhistoricalData(contract, endDateTime='', durationstr='5 Y', barSizeSetting='1 day', whatToShow= 'ADJUSTED_LAST',useRTH=True)
        if [data.date = datetime. date(my_year, my_month, my_day) for data in bars]: # If stock is actively traded THEN proceed
    #print('Lade Preisdaten für {}'. format (contract))
    values = util. tree(bars)
    for bar in values:
    ticker = bars. contract. symbol

# Strategy class Test

In [ ]:
strategy = BuyAndHold(strategy_symbol="BH",ib_client=ib,symbol="IUSQ",exchange="SMART",currency="EUR")

In [ ]:
strategy.backtest('SPY')
strategy.create_bt_summary('SPY')

In [ ]:
strategy.backtest('QQQ')
strategy.create_bt_summary('QQQ')

In [ ]:
strategy.backtest('ACWI')
strategy.create_bt_summary()

In [ ]:
strategy.df
#strategy.backtest(yf_symbol="ACWI")

In [ ]:
strategy.backtest(yf_symbol="SPY")

In [ ]:
strategy.create_bt_summary()

In [ ]:
strategy.bt_data.rename({'Adj Close':'close'})

In [ ]:
import yfinance as yf
yf.download("ACWI")


In [ ]:
contract = Stock("IUSQ", 'SMART', 'EUR')

df = util.df(ib.reqHistoricalData(contract,endDateTime='',durationStr='10 Y',barSizeSetting='1 day',whatToShow='TRADES',useRTH=True,formatDate=1))

# Create a datetime Index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Calculate the 50 Day Simple Moving Average
df["50D_SMA"] = df['close'].rolling(window=50).mean()

# Filter to keep only the last trading day of each month
month_end_df = df[df.index.day == df.index.map(last_day_of_month)].copy()

# Calculate the 10M SMA on month-end data
month_end_df['10M_SMA'] = month_end_df['close'].rolling(window=10).mean()
month_end_df['50M_SMA'] = month_end_df['close'].rolling(window=50).mean()


In [ ]:
today = dt.date.today().isoformat()

#current_price = df[df.index == today]["close"].values[0]
yesterdays_close = df[df.index < today]["close"].values[-1]
#print(f"today:{current_price} and yesterday: {yesterdays_close}")

In [ ]:
last_data = df[df.index < today].iloc[-1]
yd_close = last_data['close']
yd_close

In [ ]:
last_two_days = df[df.index < today][-2:]
last_two_days

In [ ]:
last_two_days.iloc[-2]

In [ ]:
df = bot.df

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
# Now let's plot the 10M SMA for month-end and the 50D SMA for daily data
plt.figure(figsize=(14, 7))
# Plot closing prices for the daily data
df['close'].plot(label='Close Price', color='black')
# Plot 50-day SMA for the daily data
#df['50D_SMA'].plot(label='50-Day SMA', color='red', linestyle='--')
month_end_df['50M_SMA'].plot(label='50-Month SMA', color='yellow', linestyle='--')
# Plot 10-month SMA for month-end data
plt.plot(month_end_df.index, month_end_df['10M_SMA'], label='10-Month SMA', color='red')
# Title and labels
plt.title('MSCI ACWI')
plt.xlabel('Date')
plt.ylabel('Price')

# Legend
plt.legend()

# Show the plot
plt.savefig("MSCI ACWI Condition")


# BACKTEST

In [ ]:
buy_and_hold_return

In [ ]:
ib.portfolio()

#### Helper Functions

In [ ]:
# function get_investment_weight
symbol = "IUSQ"
positions = ib.positions()

def get_investment_weight(symbol):
    "returns the investment weight in percent"
    positions = ib.positions()
    number_of_shares = [pos.position for pos in positions if pos.contract.symbol == symbol][0]
    con = [pos.contract for pos in positions if pos.contract.symbol == symbol][0]
    market_data = ib.reqMktData(con)
    latest_price = market_data.marketPrice()
    investment_value = number_of_shares * latest_price

    equitywithloan = [entry.value for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue"][0]
    
    weight = investment_value / float(equitywithloan)
    return weight *100





In [ ]:
bot = BuyAndHold(strategy_symbol="BH",ib_client=ib,symbol="IUSQ",exchange="SMART",currency="EUR")



In [ ]:
import strategies.helper_functions as hp

In [ ]:
hp.get_investment_weight(ib=ib,symbol="IUSQ")

In [ ]:
bot.fetch_data()

In [ ]:
bot.df

In [ ]:
a = get_investment_weight(ib,"IUSQ")

In [ ]:
a

In [1]:
market_data = ib.reqMktData(contract=Stock(conId=239363241, symbol='IUSQ', exchange='IBIS2', currency='EUR', localSymbol='IUSQ', tradingClass='ETF'))

NameError: name 'ib' is not defined

In [ ]:
market_data.MarketPrice()

In [ ]:
import numpy as np